In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import torch
from datasets import load_dataset, Dataset
from sentence_transformers import SentenceTransformer, CrossEncoder
import pandas as pd
import random
from tqdm import tqdm

/home/konrad-kielczynski/miniconda3/envs/golem-ner/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
# device = "mps"
device

'cuda'

In [3]:
dataset = load_dataset("sentence-transformers/squad", split="train")
dataset

Dataset({
    features: ['question', 'answer'],
    num_rows: 87599
})

In [4]:
dataset = dataset.select(range(10000))
# dataset = dataset.select(range(165, 200))
dataset

Dataset({
    features: ['question', 'answer'],
    num_rows: 10000
})

## Retrieval

In [5]:
model = SentenceTransformer("multi-qa-mpnet-base-dot-v1")
model.to(device)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

#### Calculate embeddings

In [6]:
questions_emb = model.encode(dataset["question"], convert_to_tensor=True)
questions_emb.shape

torch.Size([10000, 768])

Answer are encoded differently because they repeat in the dataset:

In [7]:
# This is very slow because we are embedding the same answer text multiple times

# dataset2 = dataset.map(
#     lambda x: {
#         "question_emb": model.encode(x["question"], convert_to_tensor=True),
#         "answer_emb": model.encode(x["answer"], convert_to_tensor=True),
#     },
#     batched=True
# )


In [8]:
df = dataset.to_pandas()

df["answer_map"] = pd.factorize(df["answer"])[0]
unique_answers = df["answer"].drop_duplicates().tolist()
print("Num of unique answers:", len(unique_answers))

dataset = Dataset.from_pandas(df)

Num of unique answers: 1867


In [9]:
answers_emb = model.encode(unique_answers, convert_to_tensor=True)
answers_emb.shape

torch.Size([1867, 768])

#### Making queries

In [10]:
top_k = 5

In [11]:
def print_query_results(question_idx, scores, indices):
    print("\nQuestion:", dataset["question"][question_idx])
    print(f"Correct answer IDX: {dataset['answer_map'][question_idx]}\n")

    for i, (score, idx) in enumerate(zip(scores, indices)):
        print(f"Answer {i+1} IDX: {idx}")
        print(f"Score: {score.item()}")
        print(f"Text: {unique_answers[idx]}\n")
    
    print("=====================================")

In [12]:
def analyze_question(question_idx, top_k=5):
    question_emb = questions_emb[question_idx]

    similarity_scores = model.similarity(question_emb, answers_emb)[0]

    scores, indices = torch.topk(similarity_scores, k=top_k)

    print_query_results(question_idx, scores, indices)

In [13]:
analyze_question(0, top_k)


Question: To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
Correct answer IDX: 0

Answer 1 IDX: 0
Score: 21.645755767822266
Text: Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.

Answer 2 IDX: 589
Score: 16.991615295410156
Text: The first documented visit by a European was in 1524 by Giovanni da Verrazzano, a Florentine explor

In [14]:
n = 5

random.seed(42)
random_indices = random.sample(range(len(dataset)), n)
random_indices

[1824, 409, 4506, 4012, 3657]

In [15]:
for idx in random_indices:
    analyze_question(idx, top_k)


Question: What condition did Frédéric describe the piano that arrived to him through many dangerous obstacles?
Correct answer IDX: 244

Answer 1 IDX: 244
Score: 23.215900421142578
Text: On 3 December, Chopin complained about his bad health and the incompetence of the doctors in Majorca: "Three doctors have visited me ... The first said I was dead; the second said I was dying; and the third said I was about to die." He also had problems having his Pleyel piano sent to him. It finally arrived from Paris in December. Chopin wrote to Pleyel in January 1839: "I am sending you my Preludes [(Op. 28)]. I finished them on your little piano, which arrived in the best possible condition in spite of the sea, the bad weather and the Palma customs." Chopin was also able to undertake work on his Ballade No. 2, Op. 38; two Polonaises, Op. 40; and the Scherzo No. 3, Op. 39.

Answer 2 IDX: 296
Score: 22.43903923034668
Text: Jones comments that "Chopin's unique position as a composer, despite the fact t

#### Add FAISS

In [16]:
answers_emb_dict = {
    "answer_emb": [emb.detach().cpu().numpy() for emb in answers_emb],
    "idx": [i for i, _ in enumerate(unique_answers)],
}

answers_emb_dataset = Dataset.from_dict(answers_emb_dict)

answers_emb_dataset

Dataset({
    features: ['answer_emb', 'idx'],
    num_rows: 1867
})

In [17]:
answers_emb_dataset.add_faiss_index(column="answer_emb")

100%|██████████| 2/2 [00:00<00:00, 245.45it/s]


Dataset({
    features: ['answer_emb', 'idx'],
    num_rows: 1867
})

In [18]:
def analyze_question_faiss(question_idx, top_k=5):
    question_emb = questions_emb[question_idx].detach().cpu().numpy()
    

    scores, samples = answers_emb_dataset.get_nearest_examples(
        "answer_emb", question_emb, k=top_k
    )

    print_query_results(question_idx, scores, samples["idx"])

In [19]:
analyze_question_faiss(0, top_k)


Question: To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
Correct answer IDX: 0

Answer 1 IDX: 0
Score: 37.27167510986328
Text: Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.

Answer 2 IDX: 589
Score: 45.78667449951172
Text: The first documented visit by a European was in 1524 by Giovanni da Verrazzano, a Florentine explorer

In [20]:
for idx in random_indices:
    analyze_question_faiss(idx)


Question: What condition did Frédéric describe the piano that arrived to him through many dangerous obstacles?
Correct answer IDX: 244

Answer 1 IDX: 217
Score: 35.46853256225586
Text: Frédéric François Chopin (/ˈʃoʊpæn/; French pronunciation: ​[fʁe.de.ʁik fʁɑ̃.swa ʃɔ.pɛ̃]; 22 February or 1 March 1810 – 17 October 1849), born Fryderyk Franciszek Chopin,[n 1] was a Polish and French (by citizenship and birth of father) composer and a virtuoso pianist of the Romantic era, who wrote primarily for the solo piano. He gained and has maintained renown worldwide as one of the leading musicians of his era, whose "poetic genius was based on a professional technique that was without equal in his generation." Chopin was born in what was then the Duchy of Warsaw, and grew up in Warsaw, which after 1815 became part of Congress Poland. A child prodigy, he completed his musical education and composed his earlier works in Warsaw before leaving Poland at the age of 20, less than a month before the outb

#### Calculate metrics (retrieval only)

In [21]:
similarity_scores = model.similarity(questions_emb, answers_emb)

_, retrieved_answers_cosine = torch.topk(similarity_scores, k=top_k)
retrieved_answers_cosine = retrieved_answers_cosine.tolist()

In [22]:
retrieved_answers_faiss = []

for i in tqdm(range(0, len(dataset))):
    question_emb = questions_emb[i].detach().cpu().numpy()

    _, samples = answers_emb_dataset.get_nearest_examples(
        "answer_emb", question_emb, k=top_k
    )

    retrieved_answers_faiss.append(samples["idx"])

100%|██████████| 10000/10000 [00:09<00:00, 1097.58it/s]


In [23]:
from metrics import calc_recall, calc_mrr

In [24]:
y_true = dataset["answer_map"]

In [25]:
recall_cosine = calc_recall(y_true, retrieved_answers_cosine, num_classes=len(unique_answers))

recall_faiss = calc_recall(y_true, retrieved_answers_faiss, num_classes=len(unique_answers))

print(f"Recall@{top_k} Cosine: ", recall_cosine)
print(f"Recall@{top_k} Faiss: ", recall_faiss)

Recall@5 Cosine:  0.9423300644189855
Recall@5 Faiss:  0.9247229096034981


In [26]:
mrr_cosine = calc_mrr(y_true, retrieved_answers_cosine)

mrr_faiss = calc_mrr(y_true, retrieved_answers_faiss)

print(f"MRR Cosine: ", mrr_cosine)
print(f"MRR Faiss: ", mrr_faiss)

MRR Cosine:  0.8072366666666587
MRR Faiss:  0.7656399999999914


## Re-ranking

In [27]:
reranker = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

In [60]:
def print_rerank_results(
    question_idx: int,
    retriever_scores: torch.Tensor,
    retriever_indices: torch.Tensor,
    reranker_scores: torch.Tensor
):
    ranking_scores, ranking = torch.sort(reranker_scores, descending=True)

    sorted_retriever_scores = retriever_scores[ranking]
    sorted_retriever_indices = retriever_indices[ranking]
    
    print("\nQuestion:", dataset["question"][question_idx])
    print(f"Correct answer IDX: {dataset['answer_map'][question_idx]}")
    print("Original order: ", retriever_indices.tolist())
    print("Reranked order: ", sorted_retriever_indices.tolist())
    print()

    for (idx, retriever_score, reranker_score) in zip(sorted_retriever_indices, sorted_retriever_scores, ranking_scores):
        print(f"Answer IDX: {idx}")
        print(f"Original score: {retriever_score.item()}")
        print(f"Reranker score: {reranker_score.item()}")
        print(f"Text: {unique_answers[idx]}\n")

In [61]:
def analyze_question_with_rerank(question_idx):
    question = dataset["question"][question_idx]
    question_emb = questions_emb[question_idx]

    similarity_scores = model.similarity(question_emb, answers_emb)[0]

    scores, indices = torch.topk(similarity_scores, k=top_k)

    reranker_input = [
        [question, unique_answers[idx]] for idx in indices
    ]
    reranker_scores = reranker.predict(reranker_input)
    reranker_scores = torch.tensor(reranker_scores)

    print_rerank_results(question_idx, scores, indices, reranker_scores)

In [62]:
analyze_question_with_rerank(0)


Question: To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
Correct answer IDX: 0
Original order:  [0, 589, 983, 38, 999]
Reranked order:  [0, 983, 38, 589, 999]

Answer IDX: 0
Original score: 21.645755767822266
Reranker score: 5.316110610961914
Text: Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.

Answer IDX: 983
Original sco

In [63]:
def analyze_question_faiss_with_rerank(question_idx):
    question = dataset["question"][question_idx]
    question_emb = questions_emb[question_idx].detach().cpu().numpy()

    scores, samples = answers_emb_dataset.get_nearest_examples(
        "answer_emb", question_emb, k=top_k
    )

    reranker_input = [
        [question, unique_answers[idx]] for idx in samples["idx"]
    ]
    reranker_scores = reranker.predict(reranker_input)

    print_rerank_results(
        question_idx, torch.tensor(scores),
        torch.tensor(samples["idx"]),
        torch.tensor(reranker_scores)
    )

In [64]:
analyze_question_faiss_with_rerank(0)


Question: To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
Correct answer IDX: 0
Original order:  [0, 589, 268, 38, 983]
Reranked order:  [0, 983, 38, 589, 268]

Answer IDX: 0
Original score: 37.27167510986328
Reranker score: 5.316109657287598
Text: Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.

Answer IDX: 983
Original scor

In [67]:
reranker_answers_faiss = []

for i, samples in tqdm(enumerate(retrieved_answers_faiss), total=len(dataset)):
    question = dataset["question"][i]

    reranker_input = [
        [question, unique_answers[idx]] for idx in samples
    ]
    reranker_scores = reranker.predict(reranker_input)

    ranking_scores, ranking = torch.sort(torch.tensor(reranker_scores), descending=True)

    retrives_indices = torch.tensor(samples)[ranking].tolist()

    reranker_answers_faiss.append(retrives_indices)



100%|██████████| 10000/10000 [02:00<00:00, 82.65it/s]


In [68]:
reranker_answers_cosine = []

for i, samples in tqdm(enumerate(retrieved_answers_cosine), total=len(dataset)):
    question = dataset["question"][i]

    reranker_input = [
        [question, unique_answers[idx]] for idx in samples
    ]
    reranker_scores = reranker.predict(reranker_input)

    ranking_scores, ranking = torch.sort(torch.tensor(reranker_scores), descending=True)

    retrives_indices = torch.tensor(samples)[ranking].tolist()

    reranker_answers_cosine.append(retrives_indices)


100%|██████████| 10000/10000 [01:51<00:00, 90.01it/s]


In [69]:
print(f'MRR@5 Retriever Cosine: {calc_mrr(y_true, retrieved_answers_cosine)}')
print(f'MRR@5 Retriever Faiss: {calc_mrr(y_true, retrieved_answers_faiss)}')
print(f'MRR@5 Reranker Cosine: {calc_mrr(y_true, reranker_answers_cosine)}')
print(f'MRR@5 Reranker Faiss: {calc_mrr(y_true, reranker_answers_faiss)}')

MRR@5 Retriever Cosine: 0.8072366666666587
MRR@5 Retriever Faiss: 0.7656399999999914
MRR@5 Reranker Cosine: 0.8697149999999981
MRR@5 Reranker Faiss: 0.8555566666666645


# Part 2 RAG

In [71]:
dataset = load_dataset("stanfordnlp/imdb")['train']
dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})

In [72]:
# encode all the texts and add faiss index

texts = dataset['text']

texts_emb = model.encode(texts, convert_to_tensor=True)

texts_emb_dict = {
    "text_emb": [emb.detach().cpu().numpy() for emb in texts_emb],
    "idx": [i for i, _ in enumerate(texts)],
}

texts_emb_dataset = Dataset.from_dict(texts_emb_dict)


texts_emb_dataset.add_faiss_index(column="text_emb")


import pickle

with open("faiss_index.pkl", "wb") as f:
    pickle.dump(texts_emb_dataset, f)

100%|██████████| 25/25 [00:00<00:00, 311.20it/s]


In [89]:
import pickle

with open("faiss_index.pkl", "rb") as f:
    texts_emb_dataset = pickle.load(f)


In [95]:
def get_top_k_retrieved_texts(question, top_k_ret=25, top_k_rerank=5):
    question_emb = model.encode(question, convert_to_tensor=True).detach().cpu().numpy()

    scores, samples = texts_emb_dataset.get_nearest_examples(
        "text_emb", question_emb, k=top_k_ret
    )

    reranker_input = [
        [question, texts[idx]] for idx in samples["idx"]
    ]

    reranker_scores = reranker.predict(reranker_input)

    ranking_scores, ranking = torch.sort(torch.tensor(reranker_scores), descending=True)

    retrives_indices = torch.tensor(samples["idx"])[ranking].tolist()

    return retrives_indices[:top_k_rerank]

In [96]:
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv("CLARIN_KEY")

In [168]:
SYSTEM_MSG_RAG_SHORT =  """
    You are a helpful assistant. Your job will be to answer questions based on the given context and not your internal knowledge.
"""

SYSTEM_MSG = """
    You are a helpful assistant. Your job will be to answer questions
"""

QUERY_INTRO = """
    Answer the following QUERY based on your internal knowledge.

    QUERY: {query}

    ANSWER:
"""

QUERY_INTRO_RAG = """
    Answer the following QUERY based on the given CONTEXT below, plese provide ANSWER to the folowing QUERY.

    QUERY: {query}

    CONTEXT: {context}

    ANSWER:
"""


In [169]:
from golemai.nlp.llm_resp_gen import LLMRespGen
import textwrap

TEXT_WRAP_WIDTH = 120

In [170]:
llm_rag = LLMRespGen(
    id_col='id',
    model_type='api',
    system_msg=SYSTEM_MSG_RAG_SHORT,
    prompt_template=QUERY_INTRO_RAG,
    batch_size=1,
    api_url='https://services.clarin-pl.eu/api/v1/oapi',
    api_key=api_key,
).set_generation_config(
    model_id='mixtral-8x22B'
)


llm_no_rag = LLMRespGen(
    id_col='id',
    model_type='api',
    system_msg=SYSTEM_MSG,
    prompt_template=QUERY_INTRO,
    batch_size=1,
    api_url='https://services.clarin-pl.eu/api/v1/oapi',
    api_key=api_key,
).set_generation_config(
    model_id='mixtral-8x22B'
)

Model type is 'api'. Setting generation config for API.
Model type is 'api'. Setting generation config for API.


In [192]:
def generate_response_no_rag(QUERY, verbose=False):
    prompts = llm_no_rag.prepare_prompt(
        query=QUERY,
    )

    if verbose:
        print("\n".join(textwrap.wrap(prompts, width=TEXT_WRAP_WIDTH)))
        print()

    result = llm_no_rag._generate_llm_response(
        inputs=prompts,
    )

    if verbose:
        print("\n".join(textwrap.wrap(result, width=TEXT_WRAP_WIDTH)))

    return result


def generate_response_rag(QUERY, top_k_ret=25, top_k_rerank=5, verbose=False):
    top_k_retrieved_texts = get_top_k_retrieved_texts(QUERY, top_k_ret, top_k_rerank)

    context = "\n".join([f"Comment [{i}]: {texts[idx]}" for i, idx in enumerate(top_k_retrieved_texts)])

    prompts = llm_rag.prepare_prompt(
        context=context,
        query=QUERY,
    )

    if verbose:
        print("\n".join(textwrap.wrap(prompts, width=TEXT_WRAP_WIDTH)))
        print()

    result = llm_rag._generate_llm_response(
        inputs=prompts,
    )

    if verbose:
        print("\n".join(textwrap.wrap(result, width=TEXT_WRAP_WIDTH)))

    return result



In [193]:
QUERY = "Who is the main character in the movie Titanic?"

In [194]:
result = generate_response_no_rag(QUERY, verbose=True)

     Answer the following QUERY based on your internal knowledge.      QUERY: Who is the main character in the movie
Titanic?      ANSWER:

 The main character in the movie Titanic is Jack Dawson, portrayed by Leonardo DiCaprio. However, the movie also has a
strong focus on Rose DeWitt Bukater, played by Kate Winslet. The story revolves around their relationship as they meet
on the ill-fated voyage of the RMS Titanic.


In [195]:
result = generate_response_rag(QUERY, verbose=True)

     Answer the following QUERY based on the given CONTEXT below, plese provide ANSWER to the folowing QUERY.
QUERY: Who is the main character in the movie Titanic?      CONTEXT: Comment [0]: I think James Cameron might be
becoming my favorite director because this is my second review of his movies. Anyway, everyone remembers the RMS
Titanic. It was big, fast, and "unsinkable"... until April 1912. It was all over the news and one of the biggest
tragedies ever. Well James Cameron decided to make a movie out of it but star two fictional characters to be in the
spotlight instead of the ship. Well, onto the main review but let me remind you that this is all opinion and zero fact
and the only fact that will be present is an event from the film.<br /><br />So our two main characters are Jack
(Leonardo DiCaprio) and Rose (Kate Winslet). They're not annoying too much but watch this and you'll find out why they
could become annoying ( http://tinyurl.com/ojhoyn ). The main villain I guess is bad

In [196]:
QUERY = "What movie you recommend to watch?"

In [197]:
result = generate_response_no_rag(QUERY, verbose=True)

     Answer the following QUERY based on your internal knowledge.      QUERY: What movie you recommend to watch?
ANSWER:

 I recommend watching "The Shawshank Redemption." It's a classic drama film that tells the story of two prisoners who
form a deep friendship and find redemption through acts of common decency. The movie is highly rated and has been
praised for its strong performances, compelling story, and thought-provoking themes. It's a great choice for anyone
looking for a powerful and moving film experience.


In [198]:
result = generate_response_rag(QUERY, verbose=True)

     Answer the following QUERY based on the given CONTEXT below, plese provide ANSWER to the folowing QUERY.
QUERY: What movie you recommend to watch?      CONTEXT: Comment [0]: for those of you who love lord of the rings and
love special effects, watch this movie! this will be sure to keep you glued to the screen. you will probably even like
it if you like watching people fight with magical stuff. Comment [1]: I'm no fan of newer movies, but this one was a
real pleasure to watch. Adults and children could watch it together - how unusual! My aunt liked it, too. It had
laughter, tears, love, adventure, special effects, good actors - and a talking parrot. It reminded me of a favourite,
The Wizard of Oz. The hero, Paulie, an intelligent parrot, is separated from his home and family and goes through many
adventures, temptations and disappointments, always keeping in mind his resolution to find his friend, Marie. Highly
recommended. Comment [2]: If you like bad movies, this is the one to s

In [199]:
QUERY = "What are the opinions about movie named Rubber?"

In [200]:
result = generate_response_no_rag(QUERY, verbose=True)

     Answer the following QUERY based on your internal knowledge.      QUERY: What are the opinions about movie named
Rubber?      ANSWER:

 "Rubber" is a 2010 French independent horror comedy film written and directed by Quentin Dupieux. The film is about a
tire that comes to life and goes on a killing spree. The movie has received mixed reviews from critics and audiences.
Some critics have praised the film for its originality, creativity, and absurd humor. They appreciate the film's unique
concept and its commentary on the nature of cinema and audience expectations. The film's technical aspects, such as its
cinematography and sound design, have also been praised.  On the other hand, some critics have found the film to be too
strange, incoherent, and self-indulgent. They argue that the film's concept is not enough to sustain a feature-length
film and that the humor is too obscure and niche. Some viewers have also found the film's pacing to be slow and its
characters to be underdevelop

In [202]:
result = generate_response_rag(QUERY, verbose=True)

     Answer the following QUERY based on the given CONTEXT below, plese provide ANSWER to the folowing QUERY.
QUERY: What are the opinions about movie named Rubber?      CONTEXT: Comment [0]: <br /><br />I watched this movie just
a little while ago and I found that this movie was terrible! It moved very slowly and was hardly entertaining!<br /><br
/>Sorry for all those that liked it.... this is only my opinion! Comment [1]: This movie is terrible but it has some
good effects. Comment [2]: I only watched this film from beginning to end because I promised a friend I would. It lacks
even unintentional entertainment value that many bad films have. It may be the worst film I have ever seen. I'm
surprised a distributor put their name on it. Comment [3]: This film is the worst film I have ever seen. The story line
is weak - I couldn't even follow it. The acting is high-schoolish. The sound track is irritating. The attempts at humor
are not. The editing is horrible. The credits are even slow -

In [203]:
QUERY = "Sumarize an opinions about movie named I Am Curious: Yellow"

In [204]:
result = generate_response_no_rag(QUERY, verbose=True)

     Answer the following QUERY based on your internal knowledge.      QUERY: Sumarize an opinions about movie named I
Am Curious: Yellow      ANSWER:

 "I Am Curious: Yellow" is a 1967 Swedish drama film directed by Vilgot Sjöman. The film is known for its explicit
sexual content and its exploration of social and political issues. The film received mixed reviews from critics and
audiences alike.  Some critics praised the film for its bold and innovative approach to filmmaking, as well as its frank
and honest portrayal of sexuality. They saw it as a groundbreaking work that challenged traditional norms and pushed the
boundaries of what was considered acceptable in cinema.  However, other critics were less impressed with the film. Some
found it to be overly explicit and gratuitous, while others felt that its political message was heavy-handed and
preachy. Some viewers also found the film to be confusing and difficult to follow, due to its non-linear narrative
structure and experimental 

In [205]:
result = generate_response_rag(QUERY, verbose=True)

     Answer the following QUERY based on the given CONTEXT below, plese provide ANSWER to the folowing QUERY.
QUERY: Sumarize an opinions about movie named I Am Curious: Yellow      CONTEXT: Comment [0]: I rented I AM CURIOUS-
YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also
heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films
considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish
drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her
attentions to making some sort of documentary on what the average Swede thought about certain political issues such as
the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm
about their opinions on politics, she has sex with her drama te

In [206]:
print("\n".join(textwrap.wrap(result, width=TEXT_WRAP_WIDTH)))

 The opinions about the movie "I Am Curious: Yellow" are varied, but overall, it is considered a thought-provoking and
engaging film that is unlike any other. Some viewers appreciate its artistic merit and its exploration of political and
social issues in Sweden, while others find it pretentious and lacking in plot. The film is known for its explicit sex
scenes, which some viewers find un-erotic and intentionally unappealing. The main character, Lena, is praised for her
realistic and human portrayal, which adds to the film's believability. However, some viewers find the film slow-paced
and difficult to follow. Overall, "I Am Curious: Yellow" is a film that requires patience and thought, and those who are
willing to invest in it will be rewarded with a unique and thought-provoking cinema experience.
